In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Experiments with sizes

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SequentialFeatureSelector

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Dissertation_final/Feature Extraction/doc2vec_features.csv')
df

,Unnamed: 0,title,page_id,doc2vec_20,doc2vec_50,doc2vec_100,doc2vec_200,doc2vec_300,doc2vec_500,quality
0,0,Mayan languages,182013,[-0.11873353 -2.4407356 -1.3702062 3.105298...,[ 0.14162295 -0.73532563 -1.8102438 -0.719203...,[ 0.21416852 -1.6696423 1.1597384 1.677749...,[-0.92931765 0.35427883 -0.985394 0.967223...,[-1.2080181 -0.70352554 -1.3096839 0.218799...,[-1.08123168e-01 -5.05179107e-01 -4.73065108e-...,1
1,1,Mu'awiya I,207068,[-2.07483 -1.4922532 0.36730346 3.651323...,[-0.94526184 -0.3110869 -1.9218819 -0.398054...,[-1.74806 -1.1632584 0.10795426 0.813521...,[-9.32636440e-01 -1.48135829e+00 1.86571255e-...,[ 0.1324497 -0.15933883 -0.56959057 0.640978...,[-1.1343669 -0.30720899 0.24345188 0.244366...,1
2,2,The Fountainhead,180464,[-0.939232 -2.142721 -1.883358 -0.108726...,[-0.89889634 -1.2338424 -0.1994613 1.074865...,[-1.2348349 -0.80121356 -0.1178944 -0.153580...,[-0.18963027 -0.5407947 0.02293755 0.160935...,[-3.95752490e-01 -2.72289246e-01 -4.50629920e-...,[ 4.51237231e-01 -3.93101126e-01 1.52836084e-...,1
3,3,Northern pintail,218361,[ 0.22895382 -0.07847051 -2.0693347 1.151979...,[-1.9152571 -0.9863042 -0.31720063 0.676733...,[-1.052102 -2.012778 0.20379645 -0.846639...,[-1.0868081 -1.1545188 -0.75203115 -0.079450...,[-0.15325828 -1.0265552 -0.950474 -0.276538...,[-2.55974084e-01 -1.00488138e+00 -7.53050864e-...,1
4,4,Manhattan Project,19603,[-0.41071716 -0.91254836 -3.3086228 0.386623...,[-0.08591695 -1.153746 -1.1509885 0.584178...,[-1.3185098 -1.2019124 0.5035422 0.113358...,[-1.0916361 -1.0232843 0.73881316 0.130395...,[ 1.85518786e-01 8.67399991e-01 -3.50051463e-...,[-2.31942884e-03 -4.61694479e-01 3.10965121e-...,1
...,...,...,...,...,...,...,...,...,...,...
5195,5195,Party of Democratic Kampuchea,265468,[-0.49925566 -0.5244077 -0.9469565 1.450050...,[-0.78056765 0.6843561 -1.1771741 0.382591...,[ 0.488909 -0.54169947 -0.3533258 0.162753...,[-2.64059007e-01 -4.53496762e-02 -6.67918384e-...,[-5.84085286e-01 6.87814727e-02 -3.03456843e-...,[-1.30619839e-01 -1.23517677e-01 2.14104261e-...,0
5196,5196,Minawara and Multultu,95240,[-0.69623053 -0.28086147 -0.49423888 0.373790...,[-0.5233514 -0.5955352 -0.45576015 -0.262624...,[-3.48152429e-01 -5.32957375e-01 -1.07259654e-...,[-0.15931535 -0.45631117 -0.14412077 0.047031...,[-0.1366955 -0.18754531 -0.12493268 -0.018171...,[-1.55988351e-01 -9.39027667e-02 -5.48811592e-...,0
5197,5197,Theophylline/ephedra/hydroxyzine,262652,[-0.55075014 -0.13917734 -0.75503784 0.880972...,[-0.5035148 -0.2839609 -0.3405831 0.169453...,[-2.67971367e-01 -4.94407803e-01 -1.63930863e-...,[-1.67261347e-01 -8.01966190e-02 -2.56586194e-...,[-1.96540684e-01 -6.82713836e-02 -1.28727436e-...,[-0.05836337 0.03332838 -0.02910833 0.038263...,0
5198,5198,"Channel Lake, Illinois",111450,[-2.5712607 -0.99652314 -2.6514347 0.483577...,[-1.2923265 -0.7806829 -1.1711309 0.212995...,[-0.6354738 -0.79841405 0.05622012 0.378250...,[-9.76243198e-01 -6.22522295e-01 -4.92107987e-...,[-0.37194127 -0.10045881 -0.15332541 -0.275343...,[-1.61765054e-01 3.33138444e-02 -8.17700550e-...,0


In [6]:
def train_and_evaluate_models(X, y, cv=5, test_size=0.2):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Initialize the models
    models = {
        'Logistic Regression': LogisticRegression(),
        'SVM': SVC(),
        'Random Forest': RandomForestClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'XGBoost': XGBClassifier(),
        'MLP': MLPClassifier(),
        'KNN': KNeighborsClassifier()
    }

    # Initialize a list to store the results
    results = []

    # Define custom scoring
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted'),
        'f1': make_scorer(f1_score, average='weighted')
    }

    # StratifiedKFold for cross-validation
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # Loop through the models
    for model_name, model in models.items():
        # Perform cross-validation on the training set
        cv_results = {metric: cross_val_score(model, X_train, y_train, cv=skf, scoring=scorer).mean() for metric, scorer in scoring.items()}

        # Train the model on the full training set
        model.fit(X_train, y_train)

        # Predict the test set
        y_pred = model.predict(X_test)

        # Calculate the metrics on the test set
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted')
        test_recall = recall_score(y_test, y_pred, average='weighted')
        test_f1 = f1_score(y_test, y_pred, average='weighted')

        # Append the results to the list
        results.append({
            'Model': model_name,
            'CV Accuracy': cv_results['accuracy'],
            'CV Precision': cv_results['precision'],
            'CV Recall': cv_results['recall'],
            'CV F1 Score': cv_results['f1'],
            'Test Accuracy': test_accuracy,
            'Test Precision': test_precision,
            'Test Recall': test_recall,
            'Test F1 Score': test_f1
        })

    # Create a dataframe from the results
    results_df = pd.DataFrame(results)

    return results_df

In [8]:
def clean_and_convert_vector(vector_str):
    # Remove newline characters and extra spaces
    cleaned_str = vector_str.replace('\n', ' ').replace('  ', ' ').strip()
    # Convert the cleaned string to a numpy array
    return np.fromstring(cleaned_str.strip('[]'), sep=' ')

In [10]:
# Size of 20
df['doc2vec_20'] = df['doc2vec_20'].apply(clean_and_convert_vector)
X = list(df['doc2vec_20'])
y = df['quality']

doc2vec_20 = train_and_evaluate_models(X, y)
doc2vec_20

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.752644,0.754487,0.752644,0.752191,0.790385,0.791944,0.790385,0.790104
1,SVM,0.801923,0.806569,0.801923,0.801177,0.820192,0.823302,0.820192,0.819759
2,Random Forest,0.793990,0.792555,0.791106,0.789554,0.821154,0.826093,0.821154,0.820474
3,AdaBoost,0.740385,0.741163,0.740385,0.740190,0.767308,0.767704,0.767308,0.767222
4,XGBoost,0.786298,0.788313,0.786298,0.785920,0.804808,0.805514,0.804808,0.804695
5,MLP,0.796394,0.800119,0.798798,0.799650,0.822115,0.822460,0.822115,0.822068
6,KNN,0.789423,0.793254,0.789423,0.788715,0.802885,0.805170,0.802885,0.802515


In [11]:
# Size of 50
df['doc2vec_50'] = df['doc2vec_50'].apply(clean_and_convert_vector)
X = list(df['doc2vec_50'])
y = df['quality']

doc2vec_50 = train_and_evaluate_models(X, y)
doc2vec_50

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.785096,0.787391,0.785096,0.784667,0.816346,0.818733,0.816346,0.816002
1,SVM,0.821875,0.827202,0.821875,0.821145,0.851923,0.854461,0.851923,0.851658
2,Random Forest,0.791106,0.801656,0.800240,0.793590,0.821154,0.826093,0.821154,0.820474
3,AdaBoost,0.756010,0.756685,0.756010,0.755855,0.781731,0.781857,0.781731,0.781706
4,XGBoost,0.807692,0.810103,0.807692,0.807325,0.822115,0.823938,0.822115,0.821865
5,MLP,0.818269,0.820765,0.813942,0.815550,0.821154,0.821325,0.821154,0.821130
6,KNN,0.811058,0.811376,0.811058,0.811008,0.829808,0.830014,0.829808,0.829781


In [12]:
# Size of 100
df['doc2vec_100'] = df['doc2vec_100'].apply(clean_and_convert_vector)
X = list(df['doc2vec_100'])
y = df['quality']

doc2vec_100 = train_and_evaluate_models(X, y)
doc2vec_100

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.793990,0.795413,0.793990,0.793744,0.816346,0.817333,0.816346,0.816203
1,SVM,0.830288,0.835745,0.830288,0.829592,0.850962,0.856532,0.850962,0.850377
2,Random Forest,0.788702,0.808409,0.792788,0.792234,0.821154,0.832486,0.821154,0.819617
3,AdaBoost,0.763702,0.763969,0.763702,0.763639,0.790385,0.790492,0.790385,0.790365
4,XGBoost,0.811298,0.815515,0.811298,0.810663,0.830769,0.832362,0.830769,0.830566
5,MLP,0.824038,0.818971,0.822115,0.815093,0.822115,0.822145,0.822115,0.822111
6,KNN,0.771635,0.796120,0.771635,0.766788,0.801923,0.823398,0.801923,0.798579


In [13]:
# Size of 200
df['doc2vec_200'] = df['doc2vec_200'].apply(clean_and_convert_vector)
X = list(df['doc2vec_200'])
y = df['quality']

doc2vec_200 = train_and_evaluate_models(X, y)
doc2vec_200

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.802885,0.803662,0.802885,0.802754,0.831731,0.832007,0.831731,0.831696
1,SVM,0.830529,0.837964,0.830529,0.829577,0.852885,0.857487,0.852885,0.852410
2,Random Forest,0.793750,0.813539,0.793990,0.791233,0.816346,0.835747,0.816346,0.813654
3,AdaBoost,0.772115,0.772576,0.772115,0.772023,0.775000,0.775016,0.775000,0.774997
4,XGBoost,0.813702,0.818908,0.813702,0.812923,0.825962,0.830817,0.825962,0.825321
5,MLP,0.819471,0.820523,0.820192,0.816289,0.823077,0.823096,0.823077,0.823074
6,KNN,0.676683,0.770309,0.676683,0.645944,0.692308,0.780875,0.692308,0.665976


In [14]:
# Size of 300
df['doc2vec_300'] = df['doc2vec_300'].apply(clean_and_convert_vector)
X = list(df['doc2vec_300'])
y = df['quality']

doc2vec_300 = train_and_evaluate_models(X, y)
doc2vec_300

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.800240,0.800909,0.800240,0.800130,0.817308,0.817425,0.817308,0.817291
1,SVM,0.830769,0.838839,0.830769,0.829761,0.853846,0.859289,0.853846,0.853291
2,Random Forest,0.791827,0.822075,0.788221,0.786760,0.801923,0.823398,0.801923,0.798579
3,AdaBoost,0.757692,0.757852,0.757692,0.757653,0.761538,0.761600,0.761538,0.761524
4,XGBoost,0.810817,0.815736,0.810817,0.810063,0.815385,0.820236,0.815385,0.814683
5,MLP,0.813221,0.823195,0.811298,0.817294,0.823077,0.823311,0.823077,0.823045
6,KNN,0.624519,0.759473,0.624519,0.568163,0.624038,0.761478,0.624038,0.567161


In [15]:
# Size of 500
df['doc2vec_500'] = df['doc2vec_500'].apply(clean_and_convert_vector)
X = list(df['doc2vec_500'])
y = df['quality']

doc2vec_500 = train_and_evaluate_models(X, y)
doc2vec_500

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.791587,0.791757,0.791587,0.791557,0.800000,0.800360,0.800000,0.799940
1,SVM,0.840865,0.848372,0.840865,0.839994,0.857692,0.862519,0.857692,0.857217
2,Random Forest,0.798798,0.827065,0.794952,0.791908,0.813462,0.844463,0.813462,0.809168
3,AdaBoost,0.762740,0.763621,0.762740,0.762549,0.768269,0.768389,0.768269,0.768243
4,XGBoost,0.816827,0.823753,0.816827,0.815829,0.832692,0.836596,0.832692,0.832206
5,MLP,0.811058,0.811399,0.808654,0.812207,0.829808,0.829907,0.829808,0.829795
6,KNN,0.570913,0.750940,0.570913,0.477186,0.574038,0.752043,0.574038,0.482704
